In [2]:
import os
import pandas as pd
import numpy as np
import data_wrangle
import json
from importlib import reload
import preprocess


#From PCA
from sklearn.decomposition import PCA
from sklearn import decomposition
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import scipy.signal as ss
import pybaselines.spline as py

import pandas as pd
import numpy as np
from math import ceil


import plotly.io as pio
pio.renderers.default = "browser"

In [20]:
reload(data_wrangle)
# db=data_wrangle.DataTable('/mnt/c/Users/16162/Desktop/Brain.db','data')
db=data_wrangle.DataTable('/home/dunn/DBM/Brain.db','data')
db.condition='WHERE strftime(\'%Y\',date)=\'2023\' AND location in (\'substantia nigra\',\'cerebral cortex\')'
# db.table_info()
db.columns('date','general_loc')[1]



Values in date:
	2022-10-12
	2022-11-03
	2022-08-15
	2022-11-10
	2023-01-10
	2023-03-30
	2023-03-01
	2023-03-03
	2023-03-16

Values in general_loc:
	blank
	center
	slice2blank
	slice2wellplatediffplateposition
	slice2wellplate
	slice2
	slice3coverslip
	subniagraish
	cortex
	midbrain
	substantia nigra
	vta
	brainstem
	tissue2
	tissue3
	532nm
	ventrical
	cerebral cortex
	subn
	low
	mb
	sn


colID,name,unique_vals_per_col,type,notnull,dflt_value,primary_key
0,id,347,INTEGER,0,None,1
1,raman_shift,11,BLOB,0,None,0
2,original_intensity,344,BLOB,0,None,0
3,frame,4,REAL,0,None,0
4,filename,188,TEXT,0,None,0
5,location,0,TEXT,0,None,0
6,slice,6,REAL,0,None,0
7,wavelength,1,TEXT,0,'532nm',0
8,power,1,TEXT,0,'10mW',0
9,objective,1,TEXT,0,'40X',0


<h1>Import to DataBase</h1> 
CSV -> BD Table


In [35]:
path='/mnt/c/Users/16162/Documents/Data/2023-03-29/'
date='2023-03-29'
operator='WG'
# slice='1'

filelist=[file for file in os.listdir(path) if '.csv' in file and 'hexane' not in file]
# for file in filelist:
#     print(file)

for file in filelist:
    name=file.split('.csv')[0]
    if 'Frame' in file:
        frame=name.split('-')[2]
        name=name.split('-F')[0]
    else:
        frame=0
    
    s=name.split('_')
    general_loc=s[0].lower()
    loc_color=s[1].lower()
    slice=s[2][-1].lower()
    # slice='1'
    pow='10mW'
    time=s[-1].split(' ')[0]
    obj='40X'
    
    df=pd.read_csv(path+file,header=None)
    x=df.iloc[:,0]
    
    x=np.array(x)
    xb=x.tobytes()
    
    y=np.array(df.iloc[:,1])
    yb=y.tobytes()

    stmt=''
    # stmt=f'raman_shift:\t\t{xb}\n'
    # stmt+=f'original_intensity:\t{yb}\n'
    stmt+=f'filename:\t\t{name}\n'
    stmt+=f'frame:\t\t\t{int(frame)}\n'
    stmt+=f'date:\t\t\t{date}\n'
    stmt+=f'slice:\t\t\t{slice}\n'
    stmt+=f'power:\t\t\t{pow}\n'
    stmt+=f'time:\t\t\t{time}\n'
    stmt+=f'operator:\t\t{operator}\n'
    stmt+=f'general_loc:\t\t{general_loc}\n'
    stmt+=f'loc_color:\t\t{loc_color}\n'
    # print(stmt)

    # db.insert_row(('raman_shift',xb),
    #               ('original_intensity',yb),
    #               ('filename',name),
    #               ('frame',int(frame)),
    #               ('date',date),('slice',slice),
    #               ('power',pow),('time',time),
    #               ('general_loc',general_loc),
    #               ('loc_color',loc_color), ('operator_name',operator)
    #               )

db.columns('date')[1]


Values in date:
	2022-10-12
	2022-11-03
	2022-08-15
	2022-11-10
	2023-01-10
	2023-03-30
	2023-03-01
	2023-03-03
	2023-03-16
	2023-03-24
	2023-03-27
	2023-03-29


colID,name,unique_vals_per_col,type,notnull,dflt_value,primary_key
0,id,539,INTEGER,0,None,1
1,raman_shift,12,BLOB,0,None,0
2,original_intensity,446,BLOB,0,None,0
3,frame,4,REAL,0,None,0
4,filename,222,TEXT,0,None,0
5,location,0,TEXT,0,None,0
6,slice,6,REAL,0,None,0
7,wavelength,1,TEXT,0,'532nm',0
8,power,1,TEXT,0,'10mW',0
9,objective,1,TEXT,0,'40X',0


<h1>Edit values in database

In [57]:
reload(data_wrangle)
db=data_wrangle.DataTable('/home/dunn/DBM/Brain.db','data')
db.columns('general_loc')[1]
# db.update('general_loc','cerebral cortex', 'cortex')


Values in general_loc:
	blank
	center
	slice2blank
	slice2wellplatediffplateposition
	slice2wellplate
	slice2
	slice3coverslip
	subniagraish
	cortex
	midbrain
	substantia nigra
	vta
	brainstem
	tissue2
	tissue3
	532nm
	ventrical
	low


colID,name,unique_vals_per_col,type,notnull,dflt_value,primary_key
0,id,539,INTEGER,0,None,1
1,raman_shift,12,BLOB,0,None,0
2,original_intensity,446,BLOB,0,None,0
3,frame,4,REAL,0,None,0
4,filename,222,TEXT,0,None,0
5,location,0,TEXT,0,None,0
6,slice,6,REAL,0,None,0
7,wavelength,1,TEXT,0,'532nm',0
8,power,1,TEXT,0,'10mW',0
9,objective,1,TEXT,0,'40X',0


<h1> PCA

In [3]:
class multivar:
    def __init__(self,intens,rs,color_by,
                 names, #color_dict,
                 hover=None,symbol=None,
                 ncomp=10):
        self.intens=intens #df
        self.rs=rs #series
        
        self.color=list(color_by)
        self.names=list(names)
        self.ncomp=ncomp

        # self.color_dict=color_dict,

        if hover==None:
            self.hover=self.names
        else:
            self.hover=hover
        if symbol==None:
            self.symbol=self.color
        else:
            self.symbol=symbol
        
        self.rs_ax='Raman Shift (cm-1)'

    
    def label_gen(self,prefix,n):
        nums=list(range(1,n+1))
        res=['%s %s'%(prefix, num) for num in nums]
        return res
    
    def symbol_gen(self,len_needed):
        syms=['circle','square','diamond','cross','x','triangle-up','star-triangle-down','triangle-left','triangle-se','pentagon','hexagon2','star','diamond-tall']
        if len_needed <= len(syms):
            return syms[0:len_needed]
        else:
            times_to_mult=ceil((len_needed/len(syms)))
            rep_syms=syms*times_to_mult
            return rep_syms[0:len_needed]

    def pca_run(self):
        pca=decomposition.PCA(n_components=self.ncomp)
        
        #Apply PCA
        X=scale(self.intens)
        X=pca.fit_transform(X)

        #Get components
        pc_labels=self.label_gen('PC',self.ncomp)
        pcs=pca.components_.T*np.sqrt(pca.explained_variance_)
        pcs=pd.DataFrame(pcs,columns=pc_labels, index=self.names)

        #Get explained variance
        explained_list=[round(x*100,2) for x in pca.explained_variance_ratio_]
        explained={pc_labels[i]:explained_list[i] for i in range(self.ncomp)}

        #Get loadings
        loadings=pd.DataFrame(X,columns=pc_labels,index=self.rs)
        return pcs, loadings, explained
    
    def pxfig(self,df,x,y,color,color_dict=None):
        syms=self.symbol_gen(len(df))
        fig=px.scatter(df,x=x,y=y,color=color,
                    #    color_discrete_map=color_dict,
                    #    color_continuous_scale=px.colors.qualitative.T10,
                    #    range_color=(-18,10),
                    #    symbol_sequence=syms, symbol=self.symbol,
                        symbol_sequence=['square','circle','diamond','cross'],
                        symbol=self.color,
                       #symbol_sequence=syms, symbol=self.names, #For color blind friendly figure
                       template='simple_white',hover_name=self.hover)
        
        # fig.update_xaxes(range=[0.8,0.9])
        fig.update_traces(marker_size=10)
        return fig

    def pca_figs(self,x,y,color=None,color_dict=None):
        if color is None:
            color=self.names

        pcs,loadings,explained=self.pca_run()

        #PCA scatter plot
        fig1=self.pxfig(pcs,x,y,color)#,color_dict=color_dict)
        fig1.update_layout(title='%s vs %s'%(x,y),font_size=20)
        fig1.update_xaxes(title_text='%s (%s%s)'%(x,explained[x],'%'),
                          range=[0.8,1])
        fig1.update_yaxes(title_text='%s (%s%s)'%(y,explained[y],'%'),
                          range=[0,0.5])
        
        # save_name='pc_zoom_mix'
        # pio.write_image(fig,'/mnt/c/Users/16162/Desktop/%s.svg'%save_name,
        #         width=1600,
        #         height=800)
        
        #PCA Loadings
        fig2=px.line(loadings,x=loadings.index,y=x,
                    title='%s (%s%s)'%(x,explained[x],'%'), template='simple_white',
                    labels={x:'','index':self.rs_ax})
        #fig.show()
        fig3=px.line(loadings,x=loadings.index,y=y,
                    title='%s (%s%s)'%(y,explained[y],'%'),template='simple_white',
                    labels={y:'','index':self.rs_ax})
        #fig.show()
        return fig1,fig2,fig3
    

    def pca_3d(self, color=None, color_dict=None,sym=None, x='PC 3', y='PC 2', z='PC 1'):
        if color is None:
            color=self.color
        if sym is None:
            pass
        pcs,loadings,explained=self.pca_run()
        
        fig=px.scatter_3d(pcs,x=x,y=y,z=z, color=color,
                        color_discrete_map=color_dict,
                        symbol=self.color, symbol_sequence=['square', 'circle', 'diamond', 'cross'],
                        # color_continuous_scale=px.colors.sequential.Jet, range_color=(0,-20),  #Jet,
                    #   color_continuous_midpoint=3,
                        
                        

                        hover_name=self.hover, 

                        )
        fig.update_layout(height=800)#, template='simple_white')
        fig.update_traces
        # fig.update_layout(scene=dict(xaxis_width=800))
        # fig.update_xaxes(showline=True, linewidth=5, linecolor='black')
        # fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
        # fig.update_zaxes(showline=True, linewidth=2, linecolor='black')
        # fig.write_html('/mnt/c/Users/16162/Desktop/3dpca.html')
        fig.show()

    def pca_fig(self,xpc,ypc):#,color_dict):
        #delete?
        figs=self.pca_figs(xpc,ypc,color=self.color)
        figs[1].show()
        figs[2].show()

    def pca_fig(self,figs):
        fig=make_subplots(rows=2,cols=2,specs=[[{'rowspan':2},{}],[None,{}]],
                          subplot_titles=[
                          figs[0].layout.title.text,figs[1].layout.title.text,figs[2].layout.title.text],
                          vertical_spacing=0.35)
        for trace in figs[0].data:
            fig.add_trace(trace,row=1,col=1)
            fig.update_xaxes(title_text=figs[0].layout.xaxis.title.text,row=1,col=1)
            fig.update_yaxes(title_text=figs[0].layout.yaxis.title.text,row=1,col=1)
        for trace in figs[1].data:
            fig.add_trace(trace,row=1,col=2)
        for trace in figs[2].data:
            fig.add_trace(trace,row=2,col=2)

        fig.update_xaxes(title_text=figs[1].layout.xaxis.title.text,row=1,col=2)
        fig.update_xaxes(title_text=figs[1].layout.xaxis.title.text,row=2,col=2)
        fig.update_xaxes(title_text=figs[0].layout.xaxis.title.text,row=1,col=1)
        fig.update_yaxes(title_text=figs[0].layout.yaxis.title.text,row=1,col=1)

        fig.update_layout(template='simple_white',
                          font_size=20,
                          font_family='ariel')
        return fig

In [58]:
os.getcwd()

'/home/dunn/spinne/DataBase'

<h1> Data Work

<h2>Get Data From DB

In [4]:
reload(data_wrangle)
db=data_wrangle.DataTable('/home/dunn/DBM/Brain.db','data') #Imported with tobytes
# db=data_wrangle.DataTable('/home/dunn/DBM/Brain.db','data')
# db.condition='WHERE strftime(\'%Y\',date)=\'2023\''
db.condition='WHERE strftime(\'%Y\',date)=\'2023\' AND general_loc in (\'substantia nigra\',\'cerebral cortex\',\'cortex\',\'midbrain\')'
# db.condition="WHERE date LIKE '2023-%'"# AND location IN ('cortex')"
# db.table_info()
x=db.raman_shifts()
ys=db.intens()
namedic=db.names()


/home/dunn/spinne/DataBase/preprocess.py:46: RuntimeWarning:

divide by zero encountered in true_divide

/home/dunn/spinne/DataBase/preprocess.py:46: RuntimeWarning:

invalid value encountered in true_divide



In [5]:
#Find NaN values
nan_cols=ys.isna().any() #Find columns with NaN values
nan_rows=ys.loc[:,nan_cols].isna().any(axis=1) #Find the rws with NaN values (in columns with NaN values)
nan_rows.index #Get indexes of rows (= sample ID)

#Drop rows with more than 5 NaN values
ys=ys.dropna(thresh=ys.shape[1]-5)

In [6]:
#Map namedic to sample ID
names=ys.index.map(lambda x: namedic[x][2])

<h1> Analysis

In [10]:
# pio.renderers.default = "browser"
var=multivar(ys.T,x,names,names)
# figs=var.pca_fig('PC 1','PC 2',ys.index)
figs=var.pca_figs('PC 2', 'PC 3')
var.pca_fig(figs)
# var.pca_fig
# var.pca_3d(color=ys.index)